In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [11]:
# Créer une session Spark
spark = SparkSession.builder.appName("Traitement_JSON").getOrCreate()

In [19]:
# Charger le fichier JSON dans un DataFrame
df = spark.read.option("multiline", "true").json('client.json')

In [20]:
# Afficher le schéma du DataFrame pour comprendre la structure
df.printSchema()

root
 |-- client_id: long (nullable = true)
 |-- nombre_appartements: long (nullable = true)
 |-- revenus: long (nullable = true)
 |-- ville: string (nullable = true)



In [21]:
df.show()

+---------+-------------------+-------+----------+
|client_id|nombre_appartements|revenus|     ville|
+---------+-------------------+-------+----------+
|        1|                  2|  50000|     Rabat|
|        2|                  1|  70000|Casablanca|
|        3|                  3|  60000|     Rabat|
|        4|                  2|  80000|Casablanca|
|        5|                  1|  55000|    Tanger|
+---------+-------------------+-------+----------+



In [22]:
# 1. Afficher les deux premières villes où l'entreprise a réalisé plus de ventes (en termes de revenus)
top_cities = (
    df.groupBy("ville")
    .agg({"revenus": "sum"})
    .orderBy(col("sum(revenus)").desc())
    .limit(2)
)
top_cities.show()

+----------+------------+
|     ville|sum(revenus)|
+----------+------------+
|Casablanca|      150000|
|     Rabat|      110000|
+----------+------------+



In [23]:
# 2. Afficher la liste des clients qui possèdent plus d'un appartement à Rabat
clients_rabat = df.filter((col("ville") == "Rabat") & (col("nombre_appartements") > 1))
clients_rabat.show()

+---------+-------------------+-------+-----+
|client_id|nombre_appartements|revenus|ville|
+---------+-------------------+-------+-----+
|        1|                  2|  50000|Rabat|
|        3|                  3|  60000|Rabat|
+---------+-------------------+-------+-----+



In [24]:
# 3. Afficher le nombre d'appartements vendus à Casablanca
appartements_casablanca = df.filter(col("ville") == "Casablanca").agg({"nombre_appartements": "sum"})
nombre_appartements_casablanca = appartements_casablanca.collect()[0][0]
print(f"Le nombre d'appartements vendus à Casablanca est : {nombre_appartements_casablanca}")


Le nombre d'appartements vendus à Casablanca est : 3


In [25]:
# Arrêter la session Spark
spark.stop()